# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_df = pd.read_csv('/Users/amber/Desktop/GitHub/python-api-challenge/output_data/cities.csv')
cities_df.dropna()
cities_df.drop(columns=['Unnamed: 0'], inplace=True)
cities_df

,City,Country,High Temp (F),Cloudiness (%),Humidity (%),Wind (mph),Latitude,Longitude
0,Bubaque,GW,78.64,3,68,21.16,11.2833,-15.8333
1,Ostrovnoy,RU,36.54,100,99,1.66,68.0531,39.5131
2,Ushuaia,AR,55.40,40,58,16.11,-54.8000,-68.3000
3,Castro,BR,74.52,18,48,5.19,-24.7911,-50.0119
4,Kapaa,US,73.40,75,73,17.27,22.0752,-159.3190
...,...,...,...,...,...,...,...,...
525,Salinas,US,55.40,90,87,5.75,36.6777,-121.6555
526,Aracati,BR,88.38,52,55,11.99,-4.5617,-37.7697
527,Bamako,ML,105.80,40,17,9.22,12.6500,-8.0000
528,Bhag,PK,89.73,0,15,3.11,29.0415,67.8239


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# set lat and long as locations
locations = cities_df[["Latitude", "Longitude"]]

# set humidity as weight
humidity = cities_df["Humidity (%)"].astype(float)

# plot the humidity heatmap
fig = gmaps.figure()

# create the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# add the head layer
fig.add_layer(heat_layer)

# display the heat map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
# narrow down cities to fit weather conditions
my_vacays = cities_df.loc[(cities_df['High Temp (F)'] <= 85) & (cities_df['Humidity (%)'] <= 40) & (cities_df['Wind (mph)'] <= 10)]
my_vacays.head()

,City,Country,High Temp (F),Cloudiness (%),Humidity (%),Wind (mph),Latitude,Longitude
10,Albany,US,68.00,62,19,8.88,42.6001,-73.9662
13,Bethel,US,68.00,1,26,8.05,41.3712,-73.4140
22,Tura,IN,78.15,65,29,4.32,25.5198,90.2201
43,Winnipeg,CA,42.01,17,24,5.75,49.8844,-97.1470
46,Jamestown,US,61.00,1,33,3.44,42.0970,-79.2353


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
# create hotel df and list for Hotel Names
hotel_df = my_vacays
hotel_df['Hotel Name'] = ''
hotels = []

params = {
    'radius': 5000,
    'types': 'lodging',
    'keyword': 'Hotel',
    'key': g_key
}

# loop through filtered dataframe for nearest hotels
for index, row in hotel_df.iterrows():
    
    lat = row['Latitude']
    lng = row['Longitude']

    params['location'] = f'{lat},{lng}'

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_name = requests.get(base_url, params=params)

    hotel_name = hotel_name.json()

    # --> NOTE: opted to go with append in order to show 'Hotel not found' result in DataFrame.
    try:
        # hotel_df.loc[index, 'Hotel Name'] = hotel_name['results'][0]['name']
        hotels.append(hotel_name['results'][0]['name'])
    except (KeyError, IndexError):
        # print('Missing field/result...skipping.')
        hotels.append('Hotel not found.')

hotel_df['Hotel Name'] = hotels
hotel_df

<ipython-input-40-4e5025f5099e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''
<ipython-input-40-4e5025f5099e>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotels


,City,Country,High Temp (F),Cloudiness (%),Humidity (%),Wind (mph),Latitude,Longitude,Hotel Name
10,Albany,US,68.00,62,19,8.88,42.6001,-73.9662,Hotel not found.
13,Bethel,US,68.00,1,26,8.05,41.3712,-73.4140,Best Western Danbury/Bethel
22,Tura,IN,78.15,65,29,4.32,25.5198,90.2201,Hotel Polo Orchid
43,Winnipeg,CA,42.01,17,24,5.75,49.8844,-97.1470,Clarion Hotel & Suites
46,Jamestown,US,61.00,1,33,3.44,42.0970,-79.2353,Chautauqua Harbor Hotel
78,Nizhniy Tsasuchey,RU,42.78,99,16,5.32,50.5167,115.1333,Hotel not found.
119,Wana,PK,66.79,0,39,1.28,32.2989,69.5725,Hotel not found.
123,Birjand,IR,73.40,40,11,4.61,32.8663,59.2211,هتل پرويز
151,Zhengjiatun,CN,43.07,0,26,6.11,43.5064,123.5064,Keyun Hotel
154,Sitka,US,53.60,1,28,3.44,57.0531,-135.3300,Totem Square Hotel & Marina


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
